In [1]:
!pip install openai google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain


  Using cached openai-1.86.0-py3-none-any.whl.metadata (25 kB)
  Using cached google_api_python_client-2.172.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_oauthlib-1.2.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
Using cached openai-1.86.0-py3-none-any.whl (730 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   -------------

In [6]:
!pip install openai google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain pytz

In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import pickle
import os

SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_google():
    creds = None
    token_path = 'token.pkl'

    # Load existing token
    if os.path.exists(token_path):
        try:
            with open(token_path, 'rb') as token_file:
                creds = pickle.load(token_file)
        except EOFError:
            print("⚠️ token.pkl is corrupted. Re-authenticating...")
            creds = None

    # Refresh or authenticate
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES
            )
            creds = flow.run_local_server(
                port=0, access_type='offline', prompt='consent'
            )

        # Save new token
        with open(token_path, 'wb') as token_file:  # ✅ Fixed here
            pickle.dump(creds, token_file)

    return creds

# Authenticate and get credentials
creds = authenticate_google()
print("✅ Authentication successful!")


RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [19]:
from datetime import datetime, timedelta
import pytz
from dateutil.parser import parse
from googleapiclient.discovery import build

def list_today_calendar_events(creds):
    service = build('calendar', 'v3', credentials=creds)

    local_tz = pytz.timezone('Africa/Lagos')
    now = datetime.now(local_tz)
    start_of_day = now.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = now.replace(hour=23, minute=59, second=59, microsecond=0)

    time_min = start_of_day.isoformat()
    time_max = end_of_day.isoformat()

    events_result = service.events().list(
        calendarId='primary',
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])

    if not events:
        print('🌸 No events scheduled for today.')
        return

    print('📅 Today\'s Events:')
    print('-------------------')

    previous_event_end = None

    for event in events:
        summary = event.get('summary', 'No Title')
        
        if 'dateTime' in event['start']:
            start_dt = parse(event['start']['dateTime'])
            end_dt = parse(event['end']['dateTime'])
            print(f"🕒 {start_dt.strftime('%H:%M')} - {end_dt.strftime('%H:%M')} | {summary}")
        else:
            print(f"📅 All Day | {summary}")
            continue

        # Meeting Preparation Reminder
        keywords = ['call', 'meeting', 'sync', 'catch-up']
        if any(keyword.lower() in summary.lower() for keyword in keywords):
            print("   ➡️ Prep needed: Fetch notes, LinkedIn bios, previous threads.")

        # Overlap detection
        if previous_event_end and start_dt < previous_event_end:
            print("   ⚠️ Conflict detected with previous event! Consider rescheduling.")

        previous_event_end = end_dt

    print('-------------------')

# Call the function
list_today_calendar_events(creds)



🌸 No events scheduled for today.


In [29]:
def calendar_agent_menu(creds):
    while True:
        print("\n📅 Calendar Agent Menu:")
        print("1. View Today's Events")
        print("2. Create New Event")
        print("3. Reschedule Event")
        print("4. Delete Event")
        print("5. Exit")
        choice = input("Enter your choice (1-5): ").strip()

        if choice == '1':
            list_today_calendar_events(creds)

        elif choice == '2':
            summary = input("Enter event summary/title: ")
            description = input("Enter event description (optional): ")
            attendees = input("Enter attendees' emails separated by comma (optional): ").split(',')

            date_input = input("Enter date (YYYY-MM-DD): ")
            start_time_input = input("Enter start time (HH:MM, AM/PM): ")
            end_time_input = input("Enter end time (HH:MM, AM/PM): ")

            local_tz = pytz.timezone('Africa/Lagos')
            try:
                start_dt = local_tz.localize(datetime.strptime(f"{date_input} {start_time_input}", "%Y-%m-%d %I:%M %p"))
                end_dt = local_tz.localize(datetime.strptime(f"{date_input} {end_time_input}", "%Y-%m-%d %I:%M %p"))
            except ValueError:
                print("⚠️ Invalid date or time format. Please use YYYY-MM-DD for date and HH:MM AM/PM for time (e.g., 01:00 PM).")
                continue

            create_calendar_event(creds, summary, start_dt, end_dt, description, [email.strip() for email in attendees if email])

        elif choice == '3':
            event_id = input("Enter the Event ID to reschedule: ")

            date_input = input("Enter new date (YYYY-MM-DD): ")
            start_time_input = input("Enter new start time (HH:MM, 24hr format): ")
            end_time_input = input("Enter new end time (HH:MM, 24hr format): ")

            local_tz = pytz.timezone('Africa/Lagos')
            try:
                new_start_dt = local_tz.localize(datetime.strptime(f"{date_input} {start_time_input}", "%Y-%m-%d %H:%M"))
                new_end_dt = local_tz.localize(datetime.strptime(f"{date_input} {end_time_input}", "%Y-%m-%d %H:%M"))
            except ValueError:
                print("⚠️ Invalid date or time format. Please use YYYY-MM-DD for date and HH:MM (24hr) for time (e.g., 13:00).")
                continue

            reschedule_calendar_event(creds, event_id, new_start_dt, new_end_dt)

        elif choice == '4':
            event_id = input("Enter the Event ID to delete: ")
            delete_calendar_event(creds, event_id)

        elif choice == '5':
            print("👋 Exiting Calendar Agent. Have a wonderful day!")
            break

        else:
            print("⚠️ Invalid choice. Please try again.")

In [33]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def create_calendar_event(creds, summary, start_dt, end_dt,
                          description='', attendees=None,
                          timezone='Africa/Lagos'):
    """
    Create a calendar event.
    - creds: authorized google credentials
    - start_dt, end_dt: timezone-aware datetime objects (pytz localized)
    - attendees: list of email strings
    - timezone: timezone string for the event (used in the event body)
    Returns the created event dict on success, None on failure.
    """
    if attendees is None:
        attendees = []

    # Basic validation
    if end_dt <= start_dt:
        print("⚠️ End time must be after start time. Event not created.")
        return None

    # Build service
    try:
        service = build('calendar', 'v3', credentials=creds)
    except Exception as e:
        print("⚠️ Failed to build calendar service:", e)
        return None

    # Prepare event body
    event_body = {
        'summary': summary,
        'description': description or '',
        'start': {
            'dateTime': start_dt.isoformat(),
            'timeZone': timezone
        },
        'end': {
            'dateTime': end_dt.isoformat(),
            'timeZone': timezone
        },
    }

    if attendees:
        # attendees should be list of emails -> convert to [{'email': ...}, ...]
        event_body['attendees'] = [{'email': email} for email in attendees if email]

    # Optional: default reminders (you can change or remove)
    event_body['reminders'] = {
        'useDefault': False,
        'overrides': [
            {'method': 'popup', 'minutes': 10},
            {'method': 'email', 'minutes': 60}
        ],
    }

    try:
        # `sendUpdates='all'` will email/calendar-invite attendees; change to 'none' to avoid.
        event = service.events().insert(
            calendarId='primary',
            body=event_body,
            sendUpdates='all'
        ).execute()
        print(f"✅ Event created: {event.get('htmlLink')}")
        return event

    except HttpError as e:
        # More detailed error handling can be added (e.status_code etc.)
        print("⚠️ Google Calendar API error:", e)
        return None
    except Exception as e:
        print("⚠️ Unexpected error creating event:", e)
        return None


In [34]:
calendar_agent_menu(creds)



📅 Calendar Agent Menu:
1. View Today's Events
2. Create New Event
3. Reschedule Event
4. Delete Event
5. Exit
⚠️ Invalid date or time format. Please use YYYY-MM-DD for date and HH:MM AM/PM for time (e.g., 01:00 PM).

📅 Calendar Agent Menu:
1. View Today's Events
2. Create New Event
3. Reschedule Event
4. Delete Event
5. Exit
✅ Event created: https://www.google.com/calendar/event?eid=MDExa3M1cmg0MW02ZWxjdDI0YjBraHN1ZW8gYWtpbnRvYmlqQG0

📅 Calendar Agent Menu:
1. View Today's Events
2. Create New Event
3. Reschedule Event
4. Delete Event
5. Exit
👋 Exiting Calendar Agent. Have a wonderful day!
